In [242]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import json
import re
import os
from time import sleep
from collections import defaultdict
import pandas as pd
from dotenv import load_dotenv
load_dotenv('.env')

True

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [41]:
root_url = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs'
years = {i:{} for i in range(2020, 1989, -1)}
for y in years:
    url = root_url.format(y)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    chart_titles = [f.text.strip() for f in soup.find_all('div', {"class": 'ye-chart-item__title'})]
    artists = [f.text.strip() for f in soup.find_all('div', {"class": 'ye-chart-item__artist'})]
    assert len(chart_titles) == len(artists)
    years[y] = {chart_titles[i]:artists[i] for i in range(len(chart_titles))}
json.dump(years, open('data/billboard-top100.json','w'), indent=2)

In [176]:
data = {}

In [106]:
def clean_str(st, lower=True, splt=[], rplc={}):
    st = st.encode("ascii", "ignore").decode()
    if lower: st = st.lower()
    for k,v in rplc.items():
        st = st.replace(k,v)
    for s in splt:
        st = st.split(s)[0]
    st = re.sub(r'[^\w\s]', '', st)
    st = re.sub('\s+',' ',st)
    return st.strip()
    
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics

In [85]:
def scrape_song_album(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    song_name = html.find('h1', class_='header_with_cover_art-primary_info-title').get_text()
    album_name = html.find('a', class_='header_with_cover_art-primary_info-primary_artist').get_text()
    return song_name, album_name

In [167]:
song_lyrics_root_url = 'https://genius.com/{}-lyrics'
for y,year_d in years.items():
#     print('.', end='')
    for i,(k,v) in enumerate(year_d.items()):
        song_key = '-'.join(clean_str(k).split(' '))
        artist_key = '-'.join(clean_str(v).split(' '))
#         artist_key = artist_key.split('-featuring')[0]
        full_key = artist_key + '-' + song_key
        if full_key in data: data[full_key]['billboard_rankings'][y] = i
        else:
            url = song_lyrics_root_url.format(full_key)
            data[full_key] = {'billboard_rankings':{y:i}, 'song': k, 'artist': v, 'genius_url':url}
            try:
                data[full_key]['lyrics'] = scrape_song_lyrics(url)
            except Exception as e:
                data[full_key]['lyrics'] = '--ERROR--'


https://genius.com/gucci-mane-and-bruno-mars-and-kodak-black-wake-up-in-the-sky-lyrics ------ Wake Up In The Sky ------ Gucci Mane X Bruno Mars X Kodak Black
wake-up-in-the-sky ------ gucci-mane-and-bruno-mars-and-kodak-black

https://genius.com/young-thug-j-cole-and-travis-scott-the-london-lyrics ------ The London ------ Young Thug, J. Cole & Travis Scott
the-london ------ young-thug-j-cole-and-travis-scott

https://genius.com/daddy-yankee-and-katy-perry-con-calma-lyrics ------ Con Calma ------ Daddy Yankee & Katy Perry Featuring Snow
con-calma ------ daddy-yankee-and-katy-perry

https://genius.com/lil-baby-and-dababy-baby-lyrics ------ Baby ------ Lil Baby & DaBaby
baby ------ lil-baby-and-dababy

https://genius.com/megan-thee-stallion-nicki-minaj-and-ty-dolla-ign-hot-girl-summer-lyrics ------ Hot Girl Summer ------ Megan Thee Stallion, Nicki Minaj & Ty Dolla $ign
hot-girl-summer ------ megan-thee-stallion-nicki-minaj-and-ty-dolla-ign

https://genius.com/pnk-walk-me-home-lyrics ----


https://genius.com/keha-tik-tok-lyrics ------ TiK ToK ------ Ke$ha
tik-tok ------ keha

https://genius.com/jayz-alicia-keys-empire-state-of-mind-lyrics ------ Empire State Of Mind ------ Jay-Z + Alicia Keys
empire-state-of-mind ------ jayz-alicia-keys

https://genius.com/keha-your-love-is-my-drug-lyrics ------ Your Love Is My Drug ------ Ke$ha
your-love-is-my-drug ------ keha

https://genius.com/fareast-movement-like-a-g6-lyrics ------ Like A G6 ------ Far*East Movement Featuring Cataracs & Dev
like-a-g6 ------ fareast-movement

https://genius.com/jayz-mr-hudson-young-forever-lyrics ------ Young Forever ------ Jay-Z + Mr. Hudson
young-forever ------ jayz-mr-hudson

https://genius.com/keha-blah-blah-blah-lyrics ------ Blah Blah Blah ------ Ke$ha Featuring 3OH!3
blah-blah-blah ------ keha

https://genius.com/keha-take-it-off-lyrics ------ Take It Off ------ Ke$ha
take-it-off ------ keha

https://genius.com/3oh3-my-first-kiss-lyrics ------ My First Kiss ------ 3OH!3 Featuring Ke$ha
my-fi


https://genius.com/sean-paul-we-be-burnin-lyrics ------ We Be Burnin' ------ Sean Paul
we-be-burnin ------ sean-paul

https://genius.com/neyo-when-youre-mad-lyrics ------ When You're Mad ------ Ne-Yo
when-youre-mad ------ neyo

https://genius.com/pnk-stupid-girls-lyrics ------ Stupid Girls ------ P!nk
stupid-girls ------ pnk

https://genius.com/tpain-im-sprung-lyrics ------ I'm Sprung ------ T-Pain
im-sprung ------ tpain

https://genius.com/neyo-so-sick-lyrics ------ So Sick ------ Ne-Yo
so-sick ------ neyo

https://genius.com/the-allamerican-rejects-move-along-lyrics ------ Move Along ------ The All-American Rejects
move-along ------ the-allamerican-rejects

https://genius.com/tpain-im-n-luv-wit-a-stripper-lyrics ------ I'm N Luv (Wit A Stripper) ------ T-Pain Featuring Mike Jones
im-n-luv-wit-a-stripper ------ tpain

https://genius.com/the-allamerican-rejects-dirty-little-secret-lyrics ------ Dirty Little Secret ------ The All-American Rejects
dirty-little-secret ------ the-allameri


https://genius.com/the-allamerican-rejects-dirty-little-secret-lyrics ------ Dirty Little Secret ------ The All-American Rejects
dirty-little-secret ------ the-allamerican-rejects

https://genius.com/neyo-sexy-love-lyrics ------ Sexy Love ------ Ne-Yo
sexy-love ------ neyo

https://genius.com/e40-u-and-dat-lyrics ------ U And Dat ------ E-40 Featuring T-Pain & Kandi Girl
u-and-dat ------ e40

https://genius.com/sean-paul-we-be-burnin-lyrics ------ We Be Burnin' ------ Sean Paul
we-be-burnin ------ sean-paul

https://genius.com/neyo-when-youre-mad-lyrics ------ When You're Mad ------ Ne-Yo
when-youre-mad ------ neyo

https://genius.com/pnk-stupid-girls-lyrics ------ Stupid Girls ------ P!nk
stupid-girls ------ pnk

https://genius.com/tpain-im-sprung-lyrics ------ I'm Sprung ------ T-Pain
im-sprung ------ tpain

https://genius.com/neyo-so-sick-lyrics ------ So Sick ------ Ne-Yo
so-sick ------ neyo

https://genius.com/the-allamerican-rejects-move-along-lyrics ------ Move Along ------ The


https://genius.com/tpain-im-n-luv-wit-a-stripper-lyrics ------ I'm N Luv (Wit A Stripper) ------ T-Pain Featuring Mike Jones
im-n-luv-wit-a-stripper ------ tpain

https://genius.com/the-allamerican-rejects-dirty-little-secret-lyrics ------ Dirty Little Secret ------ The All-American Rejects
dirty-little-secret ------ the-allamerican-rejects

https://genius.com/neyo-sexy-love-lyrics ------ Sexy Love ------ Ne-Yo
sexy-love ------ neyo

https://genius.com/e40-u-and-dat-lyrics ------ U And Dat ------ E-40 Featuring T-Pain & Kandi Girl
u-and-dat ------ e40

https://genius.com/sean-paul-we-be-burnin-lyrics ------ We Be Burnin' ------ Sean Paul
we-be-burnin ------ sean-paul

https://genius.com/neyo-when-youre-mad-lyrics ------ When You're Mad ------ Ne-Yo
when-youre-mad ------ neyo

https://genius.com/pnk-stupid-girls-lyrics ------ Stupid Girls ------ P!nk
stupid-girls ------ pnk

https://genius.com/tpain-im-sprung-lyrics ------ I'm Sprung ------ T-Pain
im-sprung ------ tpain

https://genius

In [120]:
for i,(key,key_data) in enumerate(data.items()):
    try:
        song_name, album_name = scrape_song_album(key_data['song_genius_url'])
        data[key]['song'] = song_name
        data[key]['artist'] = album_name
    except Exception as e:
        print(key)
        continue
    if album_name in key_data:
        del data[key]['album_name']
    if i%10==0: print('.', end='')

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [127]:
for i, (key,key_data) in enumerate(data.items()):
    try:
        data[key]['song_lyrics'] = scrape_song_lyrics(key_data['song_genius_url'])
    except Exception as e:
        print(key)
        continue
    if i%10==0: print('.', end='')

In [125]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [142]:
exception_keys = []
for i, (key,key_data) in enumerate(data.items()):
    artist = key_data['artist']
    song = key_data['song']
    results = spotify.search(q='artist:' + artist + ' track:' + song, type='track')
    try:
        track = results['tracks']['items'][0]
    except Exception as e:
        results = spotify.search(q=song + ' ' + artist, type='track')
        try:
            track = results['tracks']['items'][0]
        except Exception as e:
            exception_keys.append(key)
            continue
    data[key]['album_name'] = track['album']['name']
    data[key]['album_id'] = track['album']['id']
    data[key]['album_total_tracks'] = track['album']['total_tracks']
    data[key]['song_duration'] = track['duration_ms']
    data[key]['song_explicit'] = track['explicit']
    data[key]['song_id'] = track['id']
    data[key]['song_popularity'] = track['popularity']
    if i%10==0: print('.', end='')

...............................................................................................................................................

In [235]:
json.dump(data, open('data/genuis_data.json','w'), indent=2)

In [37]:
extract_feature_keys = {
    'danceability':'song_danceability',
    'energy':'song_energy',
    'key':'song_key',
    'loudness':'song_loudness',
    'mode': 'song_mode',
    'speechiness': 'song_speechiness',
    'acousticness': 'song_acousticness',
    'instrumentalness': 'song_instrumentalness',
    'liveness': 'song_liveness',
    'valence': 'song_valence',
    'tempo': 'song_tempo',
}

In [241]:
for i, (key,key_data) in enumerate(data.items()):
    feature = spotify.audio_features([key_data['song_id']])[0]
    try:
        for ex, ex_k in extract_feature_keys.items():
            data[key][ex_k] = feature[ex]
    except Exception as e:
        print(key)
        continue
    sleep(0.1)
    if i%10==0: print('.', end='')

................................................................................................................................................

In [243]:
for i, (key,key_data) in enumerate(data.items()):
    album = spotify.albums([key_data['album_id']])['albums'][0]
    data[key]['album_popularity'] = album['popularity']
    data[key]['album_genres'] = album['genres']
    if i%10==0: print('.', end='')

................................................................................................................................................

In [244]:
json.dump(data, open('data/genuis_data.json','w'), indent=2)

In [302]:
df = pd.DataFrame(data.values())
print(len(df))
df.head()

1436


,song,artist,song_billboard_rankings,song_genius_url,song_lyrics,album_id,album_total_tracks,song_duration,song_explicit,song_id,...,song_speechiness,song_acousticness,song_instrumentalness,song_liveness,song_valence,song_tempo,album_popularity,album_genres,album_name,key
0,Blinding Lights,The Weeknd,{'2020': 0},https://genius.com/the-weeknd-blinding-lights-...,Yeah\nI've been tryna call\nI've been on my ow...,4yP0hdKOZPNshxUOjY0cZj,14,200040,False,0VjIjW4GlUZAMYd2vXMi3b,...,0.0598,0.00146,0.000095,0.0897,0.334,171.005,98,[],After Hours,the-weeknd-blinding-lights
1,Circles,Post Malone,"{'2020': 1, '2019': 61}",https://genius.com/post-malone-circles-lyrics,"Oh, oh, oh\nOh, oh, oh\nOh, oh, oh, oh, oh\n...",4g1ZRSobMefqF6nelkgibi,17,215280,False,21jGcNKet2qwijlDFuPiPb,...,0.0395,0.19200,0.002440,0.0863,0.553,120.042,93,[],Hollywood's Bleeding,post-malone-circles
2,The Box,Roddy Ricch,{'2020': 2},https://genius.com/roddy-ricch-the-box-lyrics,Pullin' out the coupe at the lot\nTold 'em fuc...,52u4anZbHd6UInnmHRFzba,16,196652,True,0nbXyq5TXYPCO7pr3N8S4I,...,0.0559,0.10400,0.000000,0.7900,0.642,116.971,87,[],Please Excuse Me For Being Antisocial,roddy-ricch-the-box
3,Don’t Start Now,Dua Lipa,{'2020': 3},https://genius.com/dua-lipa-dont-start-now-lyrics,If you don't wanna see me\nDid a full one-eigh...,5iO3erM9U06Ph7MNqaxVEo,1,267253,False,7cYNQkJoyHVhP8Drb2n6d5,...,0.0569,0.00168,0.008140,0.0809,0.673,114.988,37,[],Don’t Start Now (Kaytranada Remix),dua-lipa-dont-start-now
4,ROCKSTAR,DaBaby,{'2020': 4},https://genius.com/dababy-rockstar-lyrics,"Woo, woo\nI pull up like\nHow you pull up, Bab...",623PL2MBg50Br5dLXC9E9e,13,181733,True,7ytR5pFWmSjzHJIeQkgog4,...,0.1640,0.24700,0.000000,0.1010,0.497,89.977,86,[],BLAME IT ON BABY,dababy-rockstar


In [310]:
columns_order = [
    'key', 'song', 'artist', 
    'album_id', 'album_total_tracks', 'album_popularity', 'album_genres', 'album_name',
    'song_billboard_rankings', 'song_genius_url', 'song_duration', 'song_explicit', 
    'song_id', 'song_popularity', 'song_danceability', 'song_energy', 'song_key', 'song_loudness',
    'song_mode', 'song_speechiness', 'song_acousticness', 'song_instrumentalness', 'song_liveness',
    'song_valence', 'song_tempo', 'song_lyrics'
 ]

In [311]:
df = df[columns_order]

In [316]:
df = df.drop('album_genres', axis=1)

In [328]:
billboard_rankings_range = range(2020, 1989, -1)
ranking_data = []
for i in df.iterrows():
    row = i[1]
    inner_data = {}
    for bill in billboard_rankings_range:
        inner_data['rankings'+str(bill)] = 0
        if str(bill) in row['song_billboard_rankings']:
            inner_data['rankings'+str(bill)] = row['song_billboard_rankings'][str(bill)]+1
    ranking_data.append(inner_data)

In [338]:
df = pd.concat([df, pd.DataFrame(ranking_data)], axis=1)

In [339]:
df = df.drop('song_billboard_rankings', axis=1)

In [343]:
columns_order = [
    'key', 'song', 'artist', 
    'album_id', 'album_total_tracks', 'album_popularity','album_name',
    'song_genius_url', 'song_duration', 'song_explicit', 
    'song_id', 'song_popularity', 'song_danceability', 'song_energy', 'song_key', 'song_loudness',
    'song_mode', 'song_speechiness', 'song_acousticness', 'song_instrumentalness', 'song_liveness',
    'song_valence', 'song_tempo', 
    'rankings2020', 'rankings2019', 'rankings2018', 'rankings2017', 'rankings2016', 'rankings2015', 
    'rankings2014', 'rankings2013', 'rankings2012', 'rankings2011', 'rankings2010', 'rankings2009',
    'rankings2008', 'rankings2007', 'rankings2006', 'rankings2005', 'rankings2004', 'rankings2003', 
    'rankings2002', 'rankings2001', 'rankings2000', 'rankings1999', 'rankings1998', 'rankings1997',
    'rankings1996', 'rankings1995', 'rankings1994', 'rankings1993', 'rankings1992', 'rankings1991', 
    'rankings1990',
    'song_lyrics'
 ]
df = df[columns_order]

In [344]:
change_columns_name = [
    'key', 'song', 'artist', 
    'album_id', 'album_total_tracks', 'album_popularity','album_name',
    'song_genius_url', 'song_duration', 'song_explicit', 
    'song_id', 'song_popularity', 'song_danceability', 'song_energy', 'song_key', 'song_loudness',
    'song_mode', 'song_speechiness', 'song_acousticness', 'song_instrumentalness', 'song_liveness',
    'song_valence', 'song_tempo', 
    'song_ranking_2020', 'song_ranking_2019', 'song_ranking_2018', 'song_ranking_2017', 'song_ranking_2016', 
    'song_ranking_2015', 'song_ranking_2014', 'song_ranking_2013', 'song_ranking_2012', 'song_ranking_2011', 
    'song_ranking_2010', 'song_ranking_2009','song_ranking_2008', 'song_ranking_2007', 'song_ranking_2006', 
    'song_ranking_2005', 'song_ranking_2004', 'song_ranking_2003', 'song_ranking_2002', 'song_ranking_2001', 
    'song_ranking_2000', 'song_ranking_1999', 'song_ranking_1998', 'song_ranking_1997', 'song_ranking_1996', 
    'song_ranking_1995', 'song_ranking_1994', 'song_ranking_1993', 'song_ranking_1992', 'song_ranking_1991', 
    'song_ranking_1990',
    'song_lyrics'
 ]
df.columns = change_columns_name

In [348]:
df.to_json('data/genuis_data.json', orient='records')
df.to_csv('data/genuis_data.csv', index=False)